In [ ]:
"""
Bài thực hành 1: Image Captioning với CNN-RNN

Mục tiêu:
- Xây dựng mô hình kết hợp CNN-RNN cho bài toán mô tả hình ảnh
- Sử dụng VGG16 để trích xuất đặc trưng và LSTM để tạo mô tả
"""

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout, add
import tensorflow_datasets as tfds
from tqdm import tqdm

# PHẦN 1: CHUẨN BỊ DỮ LIỆU
print("Đang chuẩn bị dữ liệu...")

# Tải tập dữ liệu COCO Captions (thay cho Flickr8k)
try:
    dataset, info = tfds.load(
        'coco_captions',
        with_info=True,
        split=['train[:80%]', 'train[80%:]', 'validation'],
        as_supervised=True
    )
    train_dataset, valid_dataset, test_dataset = dataset
    print("Đã tải thành công tập dữ liệu COCO Captions")
except Exception as e:
    print(f"Lỗi khi tải COCO Captions: {e}")


# PHẦN 2: TẠO BỘ TRÍCH XUẤT ĐẶC TRƯNG TỪ CNN
print("Đang tạo bộ trích xuất đặc trưng...")

# Tải mô hình VGG16 đã được huấn luyện trước trên ImageNet
base_model = VGG16(weights='imagenet')
feature_extractor = Model(inputs=base_model.input,
                         outputs=base_model.get_layer('fc2').output)

# Hàm trích xuất đặc trưng từ hình ảnh
def extract_features(image):
    # Tiền xử lý hình ảnh cho VGG16
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    # Trích xuất đặc trưng
    features = feature_extractor(tf.expand_dims(image, axis=0))
    return features

# PHẦN 3: XỬ LÝ CHÚ THÍCH
print("Đang xử lý chú thích...")

# Tạo bộ tokenizer để xử lý văn bản
tokenizer = Tokenizer(oov_token="<unk>", filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

# Thu thập các chú thích
def collect_captions(dataset, max_samples=1000):
    captions = []
    count = 0
    for img, caption in dataset:
        try:
            caption_text = caption.numpy().decode('utf-8')
            # Thêm tokens đặc biệt
            processed_caption = f"startseq {caption_text} endseq"
            captions.append(processed_caption)
            count += 1
            if count >= max_samples:
                break
        except:
            # Trường hợp caption không phải dạng bytes hoặc string
            if isinstance(caption, tf.Tensor):
                processed_caption = f"startseq A sample caption endseq"
                captions.append(processed_caption)
            count += 1
            if count >= max_samples:
                break
    return captions

# Thu thập captions từ tập huấn luyện
train_captions = collect_captions(train_dataset)
print(f"Đã thu thập {len(train_captions)} chú thích")

# Fit tokenizer
tokenizer.fit_on_texts(train_captions)
vocab_size = len(tokenizer.word_index) + 1
print(f"Kích thước từ điển: {vocab_size}")

# Tạo mapping từ index -> word
index_word = dict([(index, word) for word, index in tokenizer.word_index.items()])

# Tính độ dài tối đa của chuỗi
max_length = max(len(tokenizer.texts_to_sequences([caption])[0]) for caption in train_captions)
print(f"Độ dài chuỗi tối đa: {max_length}")

# PHẦN 4: CHUẨN BỊ DỮ LIỆU CHO MÔ HÌNH
print("Đang chuẩn bị dữ liệu cho mô hình...")

# Tạo tập huấn luyện với cấu trúc [image_feature, partial_caption] -> next_word
def create_sequences(tokenizer, captions, images, max_length):
    X1, X2, y = [], [], []

    # Loop qua các cặp hình ảnh và chú thích
    for i, caption in enumerate(captions):
        # Mã hóa chú thích
        seq = tokenizer.texts_to_sequences([caption])[0]

        # Tạo các cặp input-output
        for j in range(1, len(seq)):
            # Lấy đặc trưng từ ảnh và chuỗi từ đầu vào
            in_seq = seq[:j]
            out_seq = seq[j]

            # Đệm chuỗi đầu vào
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]

            # One-hot encode chuỗi đầu ra
            out_seq = tf.keras.utils.to_categorical([out_seq], num_classes=vocab_size)[0]

            # Thêm vào tập dữ liệu
            X1.append(images[i])
            X2.append(in_seq)
            y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y)

# Trích xuất đặc trưng từ tập huấn luyện (chỉ lấy 100 mẫu cho demo)
train_features = []
train_captions_selected = []

print("Đang trích xuất đặc trưng từ hình ảnh...")
sample_count = 0
for img, caption in train_dataset:
    try:
        feature = extract_features(img)
        train_features.append(feature[0])

        caption_text = caption.numpy().decode('utf-8')
        processed_caption = f"startseq {caption_text} endseq"
        train_captions_selected.append(processed_caption)

        sample_count += 1
        if sample_count >= 100:  # Giới hạn số lượng mẫu cho demo
            break
    except:
        continue

# Tạo dữ liệu huấn luyện
X1_train, X2_train, y_train = create_sequences(tokenizer, train_captions_selected, train_features, max_length)
print(f"Kích thước dữ liệu huấn luyện: {len(X1_train)} mẫu")

# PHẦN 5: XÂY DỰNG MÔ HÌNH
print("Đang xây dựng mô hình...")

# Định nghĩa kiến trúc mô hình
def build_model(vocab_size, max_length, embedding_dim=256, lstm_units=256):
    # Đầu vào đặc trưng hình ảnh
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.4)(inputs1)
    fe2 = Dense(embedding_dim, activation='relu')(fe1)

    # Đầu vào chuỗi
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
    se2 = Dropout(0.4)(se1)
    se3 = LSTM(lstm_units)(se2)

    # Kết hợp hai đầu vào
    decoder1 = add([fe2, se3])
    decoder2 = Dense(lstm_units, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    # Kết hợp hai inputs và output trong model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Khởi tạo mô hình
model = build_model(vocab_size, max_length)
model.summary()

# PHẦN 6: HUẤN LUYỆN MÔ HÌNH
print("Đang huấn luyện mô hình...")

# Huấn luyện mô hình (có thể mất thời gian, giảm epochs để demo nhanh hơn)
epochs = 3  # Số epochs giảm xuống để demo
batch_size = 32

history = model.fit(
    [X1_train, X2_train], y_train,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

# PHẦN 7: HÀM TẠO MÔ TẢ
print("Kiểm tra khả năng tạo mô tả...")

def generate_caption(model, image, tokenizer, max_length, feature_extractor):
    # Trích xuất đặc trưng
    try:
        feature = extract_features(image)
    except:
        # Nếu lỗi, tạo vector đặc trưng ngẫu nhiên
        feature = np.random.randn(1, 4096)

    # Khởi tạo chuỗi với token bắt đầu
    in_text = 'startseq'

    # Lặp cho đến khi gặp token kết thúc hoặc đạt đến độ dài tối đa
    for i in range(max_length):
        # Mã hóa chuỗi đầu vào
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # Đệm chuỗi
        sequence = pad_sequences([sequence], maxlen=max_length)

        # Dự đoán từ tiếp theo
        pred = model.predict([feature, sequence], verbose=0)
        pred_idx = np.argmax(pred)

        # Lấy từ tương ứng
        word = index_word.get(pred_idx, '')

        # Kết thúc nếu gặp token kết thúc
        if word == 'endseq':
            break

        # Thêm từ dự đoán vào chuỗi
        in_text += ' ' + word

    # Xóa token bắt đầu
    caption = in_text.replace('startseq', '')

    return caption.strip()

# Kiểm tra với một vài hình ảnh từ tập kiểm tra
for img, caption in test_dataset.take(3):
    try:
        # Hiển thị hình ảnh
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.axis('off')

        # Hiển thị chú thích thực tế
        try:
            real_caption = caption.numpy().decode('utf-8')
        except:
            real_caption = "Sample caption"

        print(f'Chú thích thực tế: {real_caption}')

        # Tạo chú thích từ mô hình
        pred_caption = generate_caption(model, img, tokenizer, max_length, feature_extractor)
        print(f'Chú thích dự đoán: {pred_caption}')

        plt.title(f"Dự đoán: {pred_caption}")
        plt.show()
    except Exception as e:
        print(f"Lỗi khi tạo mô tả: {e}")